In [1]:
from binance.websocket.spot.websocket_api import SpotWebsocketAPIClient
from binance.websocket.spot.websocket_stream import SpotWebsocketStreamClient
from binance.spot import Spot as SpotAPIClient

from common.logger import logger as logging
from trading.analize.utils.prepare_env import get_api_key

# test = False
test = True

if test:
    _api_stream_url = "wss://ws-api.testnet.binance.vision/ws-api/v3"
    _stream_url = "wss://stream.testnet.binance.vision"
    _base_url = "https://testnet.binance.vision"
else:
    _api_stream_url = "wss://ws-api.binance.com/ws-api/v3"
    _stream_url = "wss://stream.binance.com:9443"
    _base_url = "https://api.binance.com"

api_key, api_secret = get_api_key(test=test)


In [2]:
def on_close(_):
    logging.info("Do custom stuff when connection is closed")


def websocket_api_message_handler(_, message):
    logging.info("message from websocket API")
    logging.info(message)


def websocket_stream_message_handler(_, message):
    logging.info("message from websocket stream")
    logging.info(message)
    print("message from websocket stream")
    print(message)


# make a connection to the websocket api
ws_api_client = SpotWebsocketAPIClient(
    stream_url=_api_stream_url,
    api_key=api_key,
    api_secret=api_secret,
    on_message=websocket_api_message_handler,
    on_close=on_close,
)

# make a connection to the websocket stream
ws_stream_client = SpotWebsocketStreamClient(
    stream_url=_stream_url,
    on_message=websocket_stream_message_handler,
)

# spot api client to call all restful api endpoints
spot_api_client = SpotAPIClient(api_key, api_secret, base_url=_base_url)

# You can subscribe to the user data stream from websocket stream, it will broadcast all the events
# related to your account, including order updates, balance updates, etc.
# subscribe User Data stream` 
listenKey = spot_api_client.new_listen_key().get("listenKey")
ws_stream_client.user_data(listen_key=listenKey)


In [7]:
# Получение балансов
s = "-"*30
logging.info(s)

account_state = spot_api_client.account(recvWindow=50000)
balances = [asset for asset in account_state['balances'] if (float(asset['free']) != 0 or float(asset['locked']) != 0)]

logging.info(balances)
print(balances)
logging.info(s)


[{'asset': 'ETH', 'free': '2.00000000', 'locked': '0.00000000'}, {'asset': 'BTC', 'free': '2.00000000', 'locked': '0.00000000'}, {'asset': 'LTC', 'free': '8.00000000', 'locked': '0.00000000'}, {'asset': 'BNB', 'free': '2.92000000', 'locked': '0.06000000'}, {'asset': 'USDT', 'free': '19816.00000000', 'locked': '0.00000000'}, {'asset': 'TRX', 'free': '3992.00000000', 'locked': '0.00000000'}, {'asset': 'XRP', 'free': '428.00000000', 'locked': '0.00000000'}, {'asset': 'NEO', 'free': '68.00000000', 'locked': '0.00000000'}, {'asset': 'QTUM', 'free': '328.00000000', 'locked': '0.00000000'}, {'asset': 'EOS', 'free': '1250.00000000', 'locked': '0.00000000'}, {'asset': 'SNT', 'free': '24072.00000000', 'locked': '0.00000000'}, {'asset': 'GAS', 'free': '200.00000000', 'locked': '0.00000000'}, {'asset': 'LRC', 'free': '5302.00000000', 'locked': '0.00000000'}, {'asset': 'ZRX', 'free': '2148.00000000', 'locked': '0.00000000'}, {'asset': 'KNC', 'free': '1808.00000000', 'locked': '0.00000000'}, {'asset

In [6]:
# You can create a new order from websocket api
symbol = "BNBUSDT"
new_order = ws_api_client.new_order(
    symbol=symbol,
    side="SELL",
    type="LIMIT",
    timeInForce="GTC",
    quantity=0.02,
    # quoteOrderQty=11,
    price=780,
    newOrderRespType="RESULT",
)


In [ ]:
logging.info("closing ws connection")
ws_api_client.stop()
ws_stream_client.stop()